# Tax avoidance in Warren and Sanders wealth taxes

Compare the current 16% avoidance to an 8% per 1% elasticity.

For example, model 24% avoidance for families with net worth over \$1 billion who would have a 3% marginal wealth tax rate under the Warren plan, and up to 64% avoidance for families with over \$10 billion net worth for Sanders plan.

## Setup

TODO: Figure out how to make microdf work without installing labellines and taxcalc (optional).

In [1]:
import sys

In [2]:
if False:
    !pip install matplotlib-label-lines
if 'microdf' not in sys.modules:
    !pip install git+git://github.com/maxghenis/microdf.git
if 'taxcalc' not in sys.modules:
    !pip install git+git://github.com/PSLmodels/Tax-Calculator.git

  Cloning git://github.com/maxghenis/microdf.git to /tmp/pip-req-build-iutxycsz
  Running command git clone -q git://github.com/maxghenis/microdf.git /tmp/pip-req-build-iutxycsz
  Created wheel for microdf: filename=microdf-0.1-cp37-none-any.whl size=15375 sha256=acb62280173199b8455e642ed062e82d0c61c83526822b1c6bc021fe1d6e18bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-7b29emjj/wheels/d0/61/d3/083d3677aa1ee6b248d229cb6c4de037f9d61b595e9f06aed0
Successfully built microdf
  Cloning git://github.com/PSLmodels/Tax-Calculator.git to /tmp/pip-req-build-4ynyrw1m
  Running command git clone -q git://github.com/PSLmodels/Tax-Calculator.git /tmp/pip-req-build-4ynyrw1m
^C


In [3]:
import pandas as pd
import numpy as np
import microdf as mdf

## Load data

From Saez/Zucman wealth tax calculator: https://github.com/BITSS/opa-wealthtax

In [ ]:
wealth = pd.read_stata('https://github.com/BITSS/opa-wealthtax/blob/master/analysis_data/wealth.dta?raw=true',
                       columns=['networth', 'weight'])  # Don't need data source.

## Descriptive analysis

Total net worth

In [ ]:
total_nw = mdf.weighted_sum(wealth, 'networth', 'weight')
total_nw / 1e12

In [ ]:
total_families = wealth.weight.sum()
total_families / 1e6

### Warren wealth tax

2% above \$50 million, 3% above \$1 billion.

16% avoidance/evasion rate per Saez and Zucman, which is 2% * elasticity of 8: http://wealthtaxsimulator.org/analysis/

In [ ]:
AVOID_ELASTICITY = 8
CONST_AVOID_RATE = 0.16  # Originally computed as 8 * 0.02.

In [ ]:
WARREN_RATES = [0, 0.02, 0.03]  # 0%, 2%, 3%.

WARREN_RATES_M4A = [0, 0.02, 0.06]  # 0%, 2%, 6%.

WARREN_BRACKETS = [-9e99,
                   50e6,  # First $50 million.
                   1e9]   # Over $1 billion.

In [ ]:
mdf.tax_from_mtrs(wealth.networth, WARREN_BRACKETS, WARREN_RATES,
                  avoidance_rate=CONST_AVOID_RATE).sum()

In [ ]:
mdf.tax_from_mtrs(wealth.networth, WARREN_BRACKETS, WARREN_RATES,
                  avoidance_elasticity=AVOID_ELASTICITY).sum()

In [ ]:
def warren_tax(wealth, **kwargs):
    return mdf.tax_from_mtrs(wealth, WARREN_BRACKETS, WARREN_RATES, **kwargs)

### Current approach

16% constant avoidance.

In [ ]:
wealth['warren_tax'] = warren_tax(wealth.networth,
                                  avoidance_rate=CONST_AVOID_RATE)
wealth['networth_warren'] = wealth.networth - wealth.warren_tax

In [ ]:
warren_revenue = (mdf.weighted_sum(wealth, 'networth', 'weight') - 
                  mdf.weighted_sum(wealth, 'networth_warren', 'weight')).sum()
warren_revenue / 1e9

Ten-year revenue

In [ ]:
def total_wealth_tax_revenue(wealth, weight, brackets, rates, growth, **kwargs):
    # **kwargs: Pass avoidance_rate or avoidance_elasticity.
    tax = mdf.tax_from_mtrs(wealth, brackets, rates, **kwargs)
    rev = (tax * weight).sum()
    mult_10y = np.power(1 + growth, np.arange(0, 10)).sum()
    return rev * mult_10y

In [ ]:
ANNUAL_GROWTH_WARREN = 0.055
ANNUAL_GROWTH_CBO = 0.038

In [ ]:
def cartesian_product(d):
    index = pd.MultiIndex.from_product(d.values(), names=d.keys())
    return pd.DataFrame(index=index).reset_index()

In [ ]:
rev_10y = cartesian_product({'growth': [ANNUAL_GROWTH_WARREN, ANNUAL_GROWTH_CBO],
                             'avoidance': ['rate=0.15', 'elasticity=8'],
                             'plan': ['Warren', 'Warren M4A']})

In [ ]:
def wealth_tax_rev_param(growth, avoidance, plan):
    if avoidance == 'rate=0.15':
        if plan == 'Warren M4A':
            return total_wealth_tax_revenue(
                wealth.networth, wealth.weight, WARREN_BRACKETS,
                WARREN_RATES_M4A, growth, avoidance_rate=0.15)
        else:
            return total_wealth_tax_revenue(
                wealth.networth, wealth.weight, WARREN_BRACKETS,
                WARREN_RATES, growth, avoidance_rate=0.15)
    else:
        if plan == 'Warren M4A':
            return total_wealth_tax_revenue(
                wealth.networth, wealth.weight, WARREN_BRACKETS,
                WARREN_RATES_M4A, growth, avoidance_elasticity=8)
        else:
            return total_wealth_tax_revenue(
                wealth.networth, wealth.weight, WARREN_BRACKETS,
                WARREN_RATES, growth, avoidance_elasticity=8)

In [ ]:
rev_10y['rev_10y_t'] = rev_10y.apply(lambda row: wealth_tax_rev_param(
    row.growth, row.avoidance, row.plan), axis=1) / 1e12

In [ ]:
rev_10y

In [ ]:
rev_10y_wide = rev_10y.pivot_table('rev_10y_t', 'plan', ['growth', 'avoidance'])
rev_10y_wide

In [ ]:
from google.colab import files
rev_10y.to_csv('df.csv')
files.download('df.csv')

In [ ]:
rev_10y_wide.to_csv('df.csv')
files.download('df.csv')

In [ ]:
warren_10y_mult = np.power(1 + ANNUAL_GROWTH_WARREN, np.arange(0, 10)).sum()
cbo_10y_mult = np.power(1 + ANNUAL_GROWTH_CBO, np.arange(0, 10)).sum()

In [ ]:
warren_10y_mult

In [ ]:
warren_revenue * warren_10y_mult / 1e9

In [ ]:
warren_revenue * cbo_10y_mult / 1e9

### Adjusted approach

In [ ]:
wealth['warren_tax2'] = warren_tax(wealth.networth,
                                   avoidance_elasticity=AVOID_ELASTICITY)
wealth['networth_warren2'] = wealth.networth - wealth.warren_tax2

In [ ]:
warren_revenue2 = (mdf.weighted_sum(wealth, 'networth', 'weight') - 
                   mdf.weighted_sum(wealth, 'networth_warren2', 'weight')).sum()
warren_revenue2 / 1e9

In [ ]:
warren_revenue2 / warren_revenue

In [ ]:
warren_revenue2 * warren_10y_mult / 1e9

In [ ]:
warren_revenue2 * cbo_10y_mult / 1e9

## Sanders wealth tax

In [ ]:
SANDERS_RATES =    [    0, 0.01, 0.02,  0.03,  0.04, 0.05,  0.06, 0.07, 0.08]
SANDERS_BRACKETS = [-9e99, 32e6, 50e6, 250e6, 500e6,  1e9, 2.5e9,  5e9, 10e9]

In [ ]:
def sanders_tax(wealth, **kwargs):
    return mdf.tax_from_mtrs(wealth, SANDERS_BRACKETS, SANDERS_RATES, **kwargs)

### Current approach

16% constant avoidance.

In [ ]:
wealth['sanders_tax'] = sanders_tax(wealth.networth,
                                    avoidance_rate=CONST_AVOID_RATE)
wealth['networth_sanders'] = wealth.networth - wealth.sanders_tax

In [ ]:
sanders_revenue = (mdf.weighted_sum(wealth, 'networth', 'weight') - 
                   mdf.weighted_sum(wealth, 'networth_sanders', 'weight')).sum()
sanders_revenue / 1e9

### Adjusted approach

In [ ]:
wealth['sanders_tax2'] = sanders_tax(wealth.networth,
                                     avoidance_elasticity=AVOID_ELASTICITY)
wealth['networth_sanders2'] = wealth.networth - wealth.sanders_tax2

In [ ]:
sanders_revenue2 = (
    mdf.weighted_sum(wealth, 'networth', 'weight') - 
    mdf.weighted_sum(wealth, 'networth_sanders2', 'weight')).sum()
sanders_revenue2 / 1e9

In [ ]:
sanders_revenue2 / sanders_revenue